# Everything starts here

In [1]:
import numpy as np
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
import utils
from model import MobileNetRatio

## 1. Setting

### 1.1 Model parameters

In [2]:
# rebuild model
model_parameters = ('mobilenetratio', utils.num_classes, True)

### 1.2 Data augmentation

In [3]:
# data augmentation and normalization for training
# just normalization for validation
data_transforms = {
    'training': transforms.Compose([
        transforms.ToPILImage(),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'validation': transforms.Compose([
        transforms.ToPILImage(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

## 2. CNN (backbone) finetuning

### 2.0 Reload model

In [4]:
model = MobileNetRatio(*model_parameters).to(utils.device)

In [5]:
# freeze the linear layer processing the ratio
model.freeze_fc12()

### 2.1 Set training hyperparameters

In [6]:
# training parameters
LEARNING_RATE = 0.001
EPOCHS = 2
BATCH_SIZE = 64
MOMENTUM = 0.9
GAMMA = 0.1
STEP_SIZE = 1

In [7]:
# create pytorch datasets
datasets = {x: utils.HernitiaDataset(utils.dfs_path + '/' + x + '_no_temp.pkl', return_ratio_frame_idx = True, transform = data_transforms[x])  
        for x in ['training', 'validation']}

In [8]:
# instantiate data loaders
dataloaders = {x: utils.DataLoader(dataset=datasets[x], batch_size=BATCH_SIZE, shuffle=True) for x in ['training', 'validation']}

In [9]:
# criterion is cross entropy loss
criterion = nn.CrossEntropyLoss()

# observe that all parameters are being optimized
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)

# decay LR by a factor GAMMA every STEP_SIZE epochs
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

### 2.2 Train

In [10]:
utils.train_model(model = model, 
                    model_name = model.model_name,  #  name of the model which will be the name of the saved weights file within /weights
                    dataloaders = dataloaders, 
                    criterion = criterion, 
                    optimizer = optimizer, 
                    scheduler = exp_lr_scheduler, 
                    num_epochs=EPOCHS)

Epoch 1/2
----------
176692/176692: [===============================>] - ETA 1.0ssss
training Loss: 0.5223 Acc: 0.8262
38365/38365: [===============================>] - ETA 0.2sss
validation Loss: 0.6524 Acc: 0.8017
Epoch 2/2
----------
176692/176692: [===============================>] - ETA 0.4ssss
training Loss: 0.2483 Acc: 0.9186
38365/38365: [===============================>] - ETA 0.2sss
validation Loss: 0.5833 Acc: 0.8227
Training complete in 38m 47s
Best val Acc: 0.822651


## Bonus. Evaluate model

In [20]:
model = MobileNetRatio(*model_parameters).to(utils.device)

In [30]:
# reload weights from finetuning
model.load_state_dict(torch.load(utils.weights_path + '/' + model.model_name + '.pkl'))

<All keys matched successfully>

In [36]:
BATCH_SIZE = 64

# criterion is cross entropy loss
criterion = nn.CrossEntropyLoss()

In [37]:
# just normalization for validation
data_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

In [38]:
# create pytorch datasets
dataset = utils.HernitiaDataset(utils.dfs_path + '/' + 'validation' + '_lstm.pkl', data_transform)

In [39]:
# instantiate data loaders
dataloader = utils.DataLoader(dataset=dataset, batch_size=BATCH_SIZE, shuffle=False)

In [40]:
utils.evaluate_model(model, dataloader, criterion)

50464/50464: [===============================>] - ETA 0.1sss
Loss: 0.6794 Acc: 0.8016
